# Vocoders

- HifiGan (https://github.com/jik876/hifi-gan)
- BigVGAN (https://github.com/NVIDIA/BigVGAN)
 

# Autoencoder

In [1]:
from audiodiffusion.audio_encoder import AudioEncoder

audio_encoder = AudioEncoder.from_pretrained("teticio/audio-encoder")

/home/th716/.conda/envs/audiodiff_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
res = audio_encoder.encode(['../cache/spotify_sleep_dataset/waveform/34451.wav'])

In [4]:
res

tensor([[  22.3818,   55.7454,  -45.7355,   75.1192,  -23.6553,    2.2764,
           -5.6201,   25.8378,  -22.7567,   -9.3860,    6.8996,   47.9845,
          -11.0239,   49.9943,   -0.5887,   49.8197,   49.5274,   26.3963,
          -17.9764,  -40.2608,  -17.1629,   36.9612,    6.8959,   66.1374,
          -47.4549,  102.3774,  -60.5647,   41.7779,   47.6061,   46.9786,
           37.0211,   -0.6458,    7.7834,   28.1121,  -28.0773,   24.0048,
          117.4891,  -16.0868,    7.5550,   77.5549,   -2.5127,  -34.7640,
           21.7094,  -20.7481,    2.5825,   23.3617,   42.8699,  -48.1933,
           91.7696,    6.7337,    1.7964,  -20.1088,    7.7989, -119.0542,
          -14.9786,  -59.8400,   16.3396,  -26.4516,   22.9039,   29.4709,
          -40.1346,  -40.8496,  -21.7695,   51.7043,  -15.3641,  -53.3828,
          -54.8138,    4.8525,  -65.6569,  -29.4963,  -75.1400,   16.7820,
          -22.5073,  -34.0118,  -15.1272,  -40.8402,    2.3038,   -9.5547,
           -9.3951,   26.

In [6]:
len(res[0])

100

In [1]:
from audiodiffusion.utils import convert_ldm_to_hf_vae

/home/th716/.conda/envs/audiodiff_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
